In [1]:
import torch
from transformers import BertConfig,BertTokenizer,BertModel
from torch import nn
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import matplotlib.pyplot as plt
from IPython import display
from d2l import torch as d2l
import numpy as np
import pandas as pd

In [2]:
# 加载数据
excel_file = 'D:/基于深度学习的海量文本处理/第1阶段/10w.xlsx'
data_frame = pd.read_excel(excel_file)

In [3]:
# 停用词预处理
stop_words = ['您好','你好','很高兴为您服务','请问有什么可以帮您','client','user',':',' ']
def ProcessStopWords(text):
    for word in stop_words:
        text = text.replace(word,'')
    return text

data_frame['转写文本'] = data_frame['转写文本'].map(ProcessStopWords)

In [5]:
# 标签预处理
def ProcessLabels(text):
    word = '>>'
    text = text.replace(word,'>')
    return text

data_frame['服务请求'] = data_frame['服务请求'].map(ProcessLabels)

In [6]:
# 预览预处理结果
data_frame['转写文本'].iloc[:10]

0    哎我想问一下,我这个一三九的号码那个扣费方式是怎么样的扣费方式的话它是每个月呢就是每个月的一...
1    喂我那个宽带不能用宽带故障了是吗哦现在不能用前前几天就不能用了稍等我帮您看一下嗯稍等一下3天...
2    哎我我办理这个那个我想问一下我这个号码有没有开通5g套餐呀我看一下您这里有开通5g业务可以享...
3    请不要挂机您拨叫的用户正在通话中请唔好挂机您拨叫凯用户正在通话中嗯由于您多次没有声音我将结束...
4    喂我想咨询一下我的那个联通卡怎么回事啊,我都没用过嗯您可以提供一下吗这个号码就是我我号码我都...
5    嗯我想问一下我这个卡是啊升了5g上次是帮我升了5g然后现在是月租是54.5元上网费又是30一...
6                                                不要我有卡
7    我的号码为啥暂暂停服务了稍等一下我帮您看了一下的话您之前的话有反映过这个问题的是吧然后的话我...
8    哎你查一下我这个话费还有余额吗您这边还有8.95元怎么打不了怎么没有信号呢网络没有了呢呃信号...
9    哎先生哦嗯我想问一下现在如果这个号码要补号码的话那个身份证复印件有没有有效的补卡补卡要原件的...
Name: 转写文本, dtype: object

In [7]:
features = np.array(data_frame['转写文本'])
labels = np.array(data_frame['服务请求'])

In [8]:
# 加载 bert 模型
model_path = './bert-base-chinese/'
model_config = BertConfig.from_pretrained(model_path)
# 可修改模型参数

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path, config=model_config)

In [9]:
all_features = tokenizer(features.tolist(),return_tensors='pt',padding=True, truncation=True,max_length=512)
all_labels = tokenizer(labels.tolist(),return_tensors='pt',padding=True, truncation=True,max_length=100)

In [10]:
# 预览特征与标签
all_features.input_ids.shape, all_labels.input_ids.shape

(torch.Size([100000, 512]), torch.Size([100000, 75]))

In [ ]:
# 在 GPU 上训练
device = 'cuda'
model.to(device)

In [ ]:
with torch.no_grad():
    logits = model(labels_list[1]).logits
    rid = logits.argmax().item()
    model.config.id2label[rid]

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer